[View in Colaboratory](https://colab.research.google.com/github/cahya-wirawan/ML-Collection/blob/master/Text_Classification_BPPT_PANL_TPU.ipynb)

In [0]:
#!pip install textblob
#!pip install -U ntlk
#!pip install torch
#import torch

In [1]:
from sklearn import model_selection, preprocessing
from sklearn import linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import tensorflow as tf
#import pandas as pd, xgboost, numpy, textblob, string
import pandas as pd, xgboost, numpy, string
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras import layers, models, optimizers
from keras.preprocessing.sequence import pad_sequences
from pathlib import Path

import os
import numpy as np
#import ntlk

Using TensorFlow backend.


In [0]:
# Parameters
LMDATA = Path('/content/drive/My Drive/lmdata')
params = {'batch_size': 1024,
          'n_classes': 2,
          'max_len': 100,
          'n_words': 50000,
          'shuffle': True}
try:
  TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  strategy=tf.contrib.tpu.TPUDistributionStrategy(
      tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER))
except KeyError:
  TPU_WORKER = None

np.random.seed(seed=10)

In [0]:
#from google.colab import files

#uploaded = files.upload()

#for fn in uploaded.keys():
#  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!pwd
print(LMDATA)
!ls -lh "$LMDATA"
!ls -lh "$LMDATA/BPPTIndToEngCorpus"

/content
/content/drive/My Drive/lmdata
total 3.0G
-rw------- 1 root root 4.3M Apr 21 23:28 amazon-review.csv
-rw------- 1 root root  58M Oct  8 16:03 bard-1.h5
-rw------- 1 root root  32M Oct 11 07:58 bard.h5
drwx------ 2 root root 4.0K Oct  9 15:04 BPPTIndToEngCorpus
-rw------- 1 root root  25M Oct  9 16:46 wiki.id.10K.vec
-rw------- 1 root root 755M Oct  9 16:35 wiki.id.300K.vec
-rw------- 1 root root  22M Oct  8 13:31 wiki-news-300d-10K.vec
-rw------- 1 root root 2.2G Mar 14  2018 wiki-news-300d-1M.vec
total 9.2M
-rw------- 1 root root 4.6M Oct  9 15:04 bppt_panl.csv
-rw------- 1 root root 476K Oct  9 15:04 bppt_panl_test.csv
-rw------- 1 root root 4.2M Oct  9 15:04 bppt_panl_train.csv


In [0]:
train_df = pd.read_csv(LMDATA/'BPPTIndToEngCorpus/bppt_panl_train.csv')
train_df.columns = ['label', 'text']
test_df = pd.read_csv(LMDATA/'BPPTIndToEngCorpus/bppt_panl_test.csv')
test_df.columns = ['label', 'text']

In [6]:
print(train_df.head())
#print(train_df['label'][:10].values)
#print(train_df['text'][:10].values)

   label                                               text
0      3  Spurs menguasai permainan sejak saat Keane men...
1      0  Saat ini juga sedang berlangsung negosiasi den...
2      3  Pemain tengah Fenerbahce, Deivid, beralih dari...
3      0  Menurut Kepala Perwakilan BPKP di Provinsi Pap...
4      0  Indonesia memiliki potensi yang luar biasa seb...


In [7]:
!set |grep -i tpu|grep -v grep

COLAB_TPU_ADDR=10.3.206.250:8470
DATALAB_SETTINGS_OVERRIDES='{"datalabBasePath":"/tun/m/tpu-2769hgji40dub/","kernelManagerProxyPort":6000,"kernelManagerProxyHost":"172.28.0.3","jupyterArgs":["notebook","-y","--no-browser","--log-level=DEBUG","--debug","--NotebookApp.allow_origin=\"*\"","--NotebookApp.log_format=\"%(message)s\"","--NotebookApp.disable_check_xsrf=True","--NotebookApp.token=","--Session.key=\"\"","--Session.keyfile=\"\"","--ContentsManager.untitled_directory=\"Untitled Folder\"","--ContentsManager.untitled_file=\"Untitled File\"","--ContentsManager.untitled_notebook=\"Untitled Notebook\"","--KernelManager.autorestart=True","--ip=\"172.28.0.2\""]}'


In [0]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train_df['text'], train_df['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [8]:
print(train_x[:5])
print(train_y[:5])
print(valid_x[:5])
print(valid_y[:5])


19469    Pada petenis unggulan mendapatkan bye pada per...
24559    Pencetak gol terbanyak Serie A David Trezeguet...
5322     PT Pemeringkat Efek Indonesia Pefindo memberik...
15053    Gas untuk keperluan rumah tangga misalnya, mes...
20938    Kapten Bayern Munich Oliver Kahn memperingatka...
Name: text, dtype: object
[3 3 0 0 3]
7715     Pasukan dari divisi infantri ke tiga yang berm...
12394    Pemerintah dan BI akan tetap menjaga stabilita...
7327     Israel mulai melonggarkan pembatasan di pos pe...
20168    Dua pria tewas dan dua orang lainnya cedera da...
21502    Ia mengatakan meski saat ini banyak arus modal...
Name: text, dtype: object
[1 0 1 1 0]


In [0]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(train_df['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [0]:
#print(xtrain_count, xvalid_count)


In [0]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', 
                             max_features=5000)
tfidf_vect.fit(train_df['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', 
                                   ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(train_df['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}',
                                         ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(train_df['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [11]:
# load the pre-trained word-embedding vectors
max_words = params['n_words']
embeddings_index = {}
for i, line in enumerate(open('drive/My Drive/lmdata/wiki.id.10K.vec')):
  if i%10000 == 0:
    print(i)
  values = line.split()
  try:
    embeddings_index[" ".join(values[0:-300])] = numpy.asarray(values[-300:], dtype='float32')
  except ValueError:
    print("Values: {}: {}".format(i, values))

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(train_df['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), 
                                     maxlen=params['max_len'])
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), 
                                     maxlen=params['max_len'])

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    if i>=max_words:
        break
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

0
10000


In [0]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, 
                is_neural_net=False, epochs=1):
    # fit the training dataset on the classifier
    if is_neural_net:
      classifier.fit(feature_vector_train, label, epochs=epochs)
    else:
      classifier.fit(feature_vector_train, label)   
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
      predictions = [int(round(p[0])) for p in predictions]
      #predictions = predictions.argmax(axis=-1)

    print(" predictions:", predictions[:20])
    print("ground truth:", valid_y[:20])
    
    return metrics.accuracy_score(predictions, valid_y)

In [0]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("NB, CharLevel Vectors: ", accuracy)

 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
NB, Count Vectors:  0.9360279150915964
 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
NB, WordLevel TF-IDF:  0.9274498400697877
 predictions: [1 0 1 1 0 0 0 1 2 3 0 1 1 0 0 0 1 2 0 0]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
NB, N-Gram Vectors:  0.7916545507414946
 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
NB, CharLevel Vectors:  0.8685664437336436


In [0]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print( "LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR, CharLevel Vectors: ", accuracy)

 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 2 0 3 0 1 2 0 1]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
LR, Count Vectors:  0.9520209363186973
 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
LR, WordLevel TF-IDF:  0.9358825239895319
 predictions: [1 0 1 1 0 0 0 1 2 3 0 1 3 0 3 0 1 0 0 1]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
LR, N-Gram Vectors:  0.8269845885431811
 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 2 0 3 0 1 2 0 1]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
LR, CharLevel Vectors:  0.9017156150043617


In [0]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: ", accuracy)


 predictions: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
SVM, N-Gram Vectors:  0.4287583599883687


In [0]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print("RF, Count Vectors: ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print("RF, WordLevel TF-IDF: ", accuracy)

 predictions: [1 0 1 1 0 0 0 1 2 3 0 0 2 0 3 0 1 2 0 0]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
RF, Count Vectors:  0.8871765047979063
 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 2 0 3 0 2 2 0 1]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
RF, WordLevel TF-IDF:  0.883396336144228


In [0]:
# Extreme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print("Xgb, Count Vectors: ", accuracy)

# Extreme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print("Xgb, WordLevel TF-IDF: ", accuracy)

# Extreme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print("Xgb, CharLevel Vectors: ", accuracy)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 2 0 3 0 1 2 0 0]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
Xgb, Count Vectors:  0.8200058156440826


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 2 0 3 0 0 2 0 0]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
Xgb, WordLevel TF-IDF:  0.8220412910729863
 predictions: [1 0 1 1 0 0 0 1 2 3 0 2 2 0 3 0 1 2 0 1]
ground truth: [1 0 1 1 0 0 0 1 2 3 0 2 1 0 3 0 1 2 0 1]
Xgb, CharLevel Vectors:  0.8298924105844723


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [0]:
from tensorflow.keras.utils import Sequence, to_categorical

def tokenize(texts, n_words=1000):
    tokenizer = Tokenizer(num_words=n_words)
    tokenizer.fit_on_texts(texts)
    return tokenizer
  
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, texts, labels, tokenizer, batch_size=32, max_len=100,
                 n_classes=2, n_words=1000, shuffle=True):
        'Initialization'
        self.max_len = max_len
        self.batch_size = batch_size
        self.texts = texts
        self.labels = labels
        self.n_classes = n_classes
        self.shuffle = shuffle

        self.tokenizer = tokenizer
        self.steps_per_epoch = int(np.floor(self.texts.size / self.batch_size))
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return self.steps_per_epoch

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        texts = np.array([self.texts[k] for k in indexes])
        sequences = self.tokenizer.texts_to_sequences(texts)
        X = pad_sequences(sequences, maxlen=self.max_len)
        y = np.array([to_categorical(self.labels[k], 4) for k in indexes])

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.texts.size)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

In [0]:
# Create data generator
training_generator = DataGenerator(train_x.values, train_y, token, **params)
valid_generator = DataGenerator(valid_x.values, valid_y, token,  **params)

In [0]:
def tpu_wrapper(func):
  if TPU_WORKER is not None:
    tpu_model = tf.contrib.tpu.keras_to_tpu_model(func, strategy)
    return tpu_model
  else:
    return func

In [0]:
# This model doesn't work with current dataset
def create_simple_model(input_length=100):
    # create input layer 
    input_layer = layers.Input((input_length, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu", name="D1")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(4, activation="softmax")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy')
    return classifier 

#classifier = create_simple_model(xtrain_tfidf_ngram.shape[1])
#accuracy = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, 
#                       is_neural_net=True, epochs=1)
#print("NN, Ngram Level TF IDF Vectors",  accuracy)

In [0]:
%%time

#NN, Ngram Level TF IDF Vectors
tf.keras.backend.clear_session()
classifier = create_simple_model(input_length=params['max_len'])
classifier = tpu_wrapper(classifier)
classifier.fit_generator(
    generator=training_generator,
    validation_data=valid_generator,
    #use_multiprocessing=True,
    #workers=6,
    epochs=20
)

classifier.save_weights(str(LMDATA/'bard.h5'), overwrite=True)

In [22]:
xtrain_tfidf_ngram.shape

(20631, 5000)

In [0]:
"""
def to_tpu(func):
    def wrapper(*args, **kwargs):
      print("TPU Wrapper start")
      print(func)
      if TPU_WORKER is not None:
        tpu_model = tf.contrib.tpu.keras_to_tpu_model(func, strategy)
        print("TPU exist")
        return tpu_model(*args, **kwargs)
      else:
        print("TPU not exist")
        return func(*args, **kwargs)
    print("TO_TPU")
    return wrapper
"""

In [0]:
def create_cnn(input_length=100):
    # Add an Input Layer
    input_layer = layers.Input((input_length, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, 
                                       weights=[embedding_matrix], 
                                       trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(100, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.3)(output_layer1)
    output_layer2 = layers.Dense(4, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [0]:
# Create data generator
training_generator = DataGenerator(train_x.values, train_y, token, **params)
valid_generator = DataGenerator(valid_x.values, valid_y, token,  **params)


In [0]:
%%time

tf.keras.backend.clear_session()
classifier = create_cnn(input_length=params['max_len'])
classifier = tpu_wrapper(classifier)
classifier.fit_generator(
    generator=training_generator,
    validation_data=valid_generator,
    #use_multiprocessing=True,
    #workers=6,
    epochs=20
)

classifier.save_weights(str(LMDATA/'bard.h5'), overwrite=True)


INFO:tensorflow:Connecting to: b'grpc://10.3.206.250:8470'
Epoch 1/20
INFO:tensorflow:New input shapes; (re-)compiling: mode=train, [TensorSpec(shape=(128, 100), dtype=tf.float32, name='input_10'), TensorSpec(shape=(128, 4), dtype=tf.float32, name='dense_1_target_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
INFO:tensorflow:Cloning Adam {'lr': 0.0010000000474974513, 'beta_1': 0.8999999761581421, 'beta_2': 0.9990000128746033, 'decay': 0.0, 'epsilon': 1e-07, 'amsgrad': False}
INFO:tensorflow:Get updates: Tensor("loss/mul:0", shape=(), dtype=float32)
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 3.7215137481689453 secs
INFO:tensorflow:Setting weights on TPU model.
19/20 [===========================>..] - ETA: 0s - loss: 0.9698 - acc: 0.6209INFO:tensorflow:New input shapes; (re-)compiling: mode=eval, [TensorSpec(shape=(128, 100), dtype=tf.float32, name='input_10'), TensorSpec(shape=(128, 4), dty

In [0]:

classifier.save_weights(str(LMDATA/'bard.h5'), overwrite=True)

INFO:tensorflow:Copying TPU weights to the CPU


In [0]:
# predict the labels on test dataset


sequences = token.texts_to_sequences(test_df['text'].values)
test_x_seq = pad_sequences(sequences, maxlen=params['max_len'])
#print(valid_x_seq[:5])
classifier = create_cnn(input_length=params['max_len'])
classifier.load_weights(str(LMDATA/'bard.h5'))
# TPU can be enabled here if we need  
# classifier = tpu_wrapper(classifier)
labels = np.array([list(to_categorical(label, 4).astype(int)) for label in test_df['label'].values])
score = classifier.evaluate(test_x_seq, labels, verbose=1)
print('Loss for final step: {}, accuracy: {}'.format(score[0], score[1]))




3056/3056 [==============================] - 1s 487us/step
Loss for final step: 0.18855890020221003, accuracy: 0.9355366492146597


In [0]:
"""
# Just for testing
labels = [list(to_categorical(label, 4).astype(int)) for label in test_df['label'].values]
#print(labels[:5])
print(np.array(labels)[:5])
print(test_df['label'].values)
print(to_categorical(3, 4))
"""

In [0]:
prediction = classifier.predict(test_x_seq, verbose=1)
#print(prediction)
print('Predictions for final step: {}'.format(np.argmax(score, axis=1)[:10]))
print(valid_x[:10])

3056/3056 [==============================] - 2s 601us/step
Predictions for final step: [2 1 0 0 0 1 2 0 2 0]
7715     Pasukan dari divisi infantri ke tiga yang berm...
12394    Pemerintah dan BI akan tetap menjaga stabilita...
7327     Israel mulai melonggarkan pembatasan di pos pe...
20168    Dua pria tewas dan dua orang lainnya cedera da...
21502    Ia mengatakan meski saat ini banyak arus modal...
19559    Agri Resources BV, sebuah perusahaan di Beland...
6336     Asalkan kita punya program-program yang bagus ...
2881     Sikap ini diperkuat oleh hasil interogasi terh...
16877            Gurita juga menggunakan sistem yang sama.
26868    Tantangan utamanya mungkin akan datang dari Ra...
Name: text, dtype: object


In [0]:
!free

              total        used        free      shared  buff/cache   available
Mem:       13335212     1220196     8944632         804     3170384    12348052
Swap:             0           0           0


In [0]:

#print('Accuracy ', score[1])
"""
# First, run the seed forward to prime the state of the model.
#prediction_model.reset_states()
strategy = tf.contrib.tpu.TPUDistributionStrategy(
    tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER))
prediction_model = tf.contrib.tpu.keras_to_tpu_model(
    prediction_model, strategy=strategy)

predictions = prediction_model.predict(valid_x_seq)

print("predictions", predictions[:20])

predictions = tpu_model.predict(valid_x_seq)

predictions = [int(round(p[0])) for p in predictions]
#predictions = predictions.argmax(axis=-1)

print("predictions", predictions[:20])
print("valid_y", valid_y[:20])

return metrics.accuracy_score(predictions, valid_y)
"""

In [0]:
print(score)
print(np.argmax(score, axis=1))


[[8.6630200e-04 1.0124649e-03 9.9810225e-01 1.8999004e-05]
 [3.0715969e-06 9.9983561e-01 1.4403675e-04 1.7290302e-05]
 [9.5167154e-01 1.1388691e-02 3.6110844e-02 8.2881597e-04]
 ...
 [1.1887937e-03 1.5343540e-02 9.8264515e-01 8.2238560e-04]
 [9.9969316e-01 4.9298887e-07 3.0627038e-04 8.0982971e-08]
 [7.7077589e-04 3.7112991e-06 9.9922478e-01 7.3053889e-07]]
[2 1 0 ... 2 0 2]


# New Section

In [0]:
def create_rnn_lstm(input_length=100):
    # Add an Input Layer
    input_layer = layers.Input((input_length, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.2)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)
    #lstm_layer = layers.LSTM(100)(lstm_layer)
    #lstm_layer = layers.LSTM(100, return_sequences=True)(lstm_layer)
    #lstm_layer = layers.TimeDistributed(layers.Dense(100))(lstm_layer)
    # Add the output Layers
    output_layer1 = layers.Dense(100, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.2)(output_layer1)
    output_layer2 = layers.Dense(4, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [0]:
%%time

# RNN-LSTM, Word Embeddings
# Speed comparison between CPU vs TPU
# First we test CPU with 2 epochs
tf.keras.backend.clear_session()
classifier = create_rnn_lstm(input_length=params['max_len'])
#classifier = tf.contrib.tpu.keras_to_tpu_model(classifier, strategy=strategy)
classifier.fit_generator(
    generator=training_generator,
    validation_data=valid_generator,
    #use_multiprocessing=True,
    #workers=6,
    epochs=2
)

Epoch 1/2
20/20 [==============================] - 79s 4s/step - loss: 1.1150 - acc: 0.5111 - val_loss: 0.6440 - val_acc: 0.8319
Epoch 2/2
20/20 [==============================] - 77s 4s/step - loss: 0.5015 - acc: 0.8368 - val_loss: 0.3377 - val_acc: 0.8830
CPU times: user 4min 50s, sys: 10.2 s, total: 5min
Wall time: 2min 37s


In [0]:
%%time

# RNN-LSTM, Word Embeddings
# We test now the TPU
# The result is:
# CPU: 77s/epoch
# TPU: 3.15s/epoch
tf.keras.backend.clear_session()
classifier = create_rnn_lstm(input_length=params['max_len'])
classifier = tpu_wrapper(classifier)
classifier.fit_generator(
    generator=training_generator,
    validation_data=valid_generator,
    #use_multiprocessing=True,
    #workers=6,
    epochs=20
)

Epoch 1/20
INFO:tensorflow:New input shapes; (re-)compiling: mode=train, [TensorSpec(shape=(128, 100), dtype=tf.float32, name='input_10'), TensorSpec(shape=(128, 4), dtype=tf.float32, name='dense_1_target_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
INFO:tensorflow:Cloning Adam {'lr': 0.0010000000474974513, 'beta_1': 0.8999999761581421, 'beta_2': 0.9990000128746033, 'decay': 0.0, 'epsilon': 1e-07, 'amsgrad': False}
INFO:tensorflow:Get updates: Tensor("loss_6/mul:0", shape=(), dtype=float32)
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 5.248517751693726 secs
INFO:tensorflow:Setting weights on TPU model.
19/20 [===========================>..] - ETA: 0s - loss: 1.0701 - acc: 0.5391INFO:tensorflow:New input shapes; (re-)compiling: mode=eval, [TensorSpec(shape=(128, 100), dtype=tf.float32, name='input_10'), TensorSpec(shape=(128, 4), dtype=tf.float32, name='dense_1_target_10')]
INFO:tensorflow:

In [0]:
def create_rnn_gru(input_length=100):
    # Add an Input Layer
    input_layer = layers.Input((input_length, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(4, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [0]:
%%time

# RNN-GRU, Word Embeddings
tf.keras.backend.clear_session()
classifier = create_rnn_gru(input_length=params['max_len'])
classifier = tpu_wrapper(classifier)
classifier.fit_generator(
    generator=training_generator,
    validation_data=valid_generator,
    #use_multiprocessing=True,
    #workers=6,
    epochs=20
)

Epoch 1/20
INFO:tensorflow:New input shapes; (re-)compiling: mode=train, [TensorSpec(shape=(128, 100), dtype=tf.float32, name='input_10'), TensorSpec(shape=(128, 4), dtype=tf.float32, name='dense_1_target_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
INFO:tensorflow:Cloning Adam {'lr': 0.0010000000474974513, 'beta_1': 0.8999999761581421, 'beta_2': 0.9990000128746033, 'decay': 0.0, 'epsilon': 1e-07, 'amsgrad': False}
INFO:tensorflow:Get updates: Tensor("loss_8/mul:0", shape=(), dtype=float32)
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 6.248059988021851 secs
INFO:tensorflow:Setting weights on TPU model.
19/20 [===========================>..] - ETA: 0s - loss: 1.1141 - acc: 0.5259INFO:tensorflow:New input shapes; (re-)compiling: mode=eval, [TensorSpec(shape=(128, 100), dtype=tf.float32, name='input_10'), TensorSpec(shape=(128, 4), dtype=tf.float32, name='dense_1_target_10')]
INFO:tensorflow:

In [0]:
# RNN-Bidirectional, Word Embeddings
# It doesn't work with TPU, but it works on CPU/GPU
def create_bidirectional_rnn(input_length=100):
    # Add an Input Layer
    input_layer = layers.Input((input_length, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(4, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [26]:
%%time

# RNN-Bidirectional, Word Embeddings
tf.keras.backend.clear_session()
classifier = create_bidirectional_rnn(input_length=params['max_len'])
classifier = tpu_wrapper(classifier)
classifier.fit_generator(
    generator=training_generator,
    validation_data=valid_generator,
    #use_multiprocessing=True,
    #workers=6,
    epochs=20
)

Epoch 1/20
20/20 [==============================] - 19s 973ms/step - loss: 1.2091 - acc: 0.4466 - val_loss: 0.9484 - val_acc: 0.6694
Epoch 2/20
20/20 [==============================] - 14s 717ms/step - loss: 0.7157 - acc: 0.7386 - val_loss: 0.4386 - val_acc: 0.8530
Epoch 3/20
20/20 [==============================] - 15s 738ms/step - loss: 0.4169 - acc: 0.8549 - val_loss: 0.3147 - val_acc: 0.8856
Epoch 4/20
20/20 [==============================] - 14s 711ms/step - loss: 0.3404 - acc: 0.8826 - val_loss: 0.2709 - val_acc: 0.9048
Epoch 5/20
20/20 [==============================] - 15s 755ms/step - loss: 0.3073 - acc: 0.8938 - val_loss: 0.2566 - val_acc: 0.9069
Epoch 6/20
20/20 [==============================] - 15s 733ms/step - loss: 0.2938 - acc: 0.8962 - val_loss: 0.2484 - val_acc: 0.9116
Epoch 7/20
20/20 [==============================] - 15s 751ms/step - loss: 0.2748 - acc: 0.9028 - val_loss: 0.2504 - val_acc: 0.9118
Epoch 8/20
20/20 [==============================] - 15s 734ms/step - 

In [0]:
def create_rcnn(input_length=100):
    # Add an Input Layer
    input_layer = layers.Input((input_length, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
    
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(4, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

#classifier = create_rcnn()
#accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, 
#                       is_neural_net=True, epochs=10)
#print("CNN, Word Embeddings",  accuracy)

In [28]:
%%time

# RCNN, Word Embeddings
tf.keras.backend.clear_session()
classifier = create_bidirectional_rnn(input_length=params['max_len'])
classifier = tpu_wrapper(classifier)
classifier.fit_generator(
    generator=training_generator,
    validation_data=valid_generator,
    #use_multiprocessing=True,
    #workers=6,
    epochs=20
)

Epoch 1/20
20/20 [==============================] - 20s 1s/step - loss: 1.1720 - acc: 0.4778 - val_loss: 0.8795 - val_acc: 0.6781
Epoch 2/20
20/20 [==============================] - 15s 737ms/step - loss: 0.6697 - acc: 0.7627 - val_loss: 0.3682 - val_acc: 0.8750
Epoch 3/20
20/20 [==============================] - 15s 767ms/step - loss: 0.3951 - acc: 0.8652 - val_loss: 0.2989 - val_acc: 0.8958
Epoch 4/20
20/20 [==============================] - 14s 712ms/step - loss: 0.3331 - acc: 0.8842 - val_loss: 0.2719 - val_acc: 0.9027
Epoch 5/20
20/20 [==============================] - 15s 761ms/step - loss: 0.3086 - acc: 0.8920 - val_loss: 0.2577 - val_acc: 0.9085
Epoch 6/20
20/20 [==============================] - 15s 726ms/step - loss: 0.2913 - acc: 0.8978 - val_loss: 0.2602 - val_acc: 0.9106
Epoch 7/20
20/20 [==============================] - 15s 754ms/step - loss: 0.2788 - acc: 0.9010 - val_loss: 0.2493 - val_acc: 0.9116
Epoch 8/20
20/20 [==============================] - 14s 718ms/step - los

# New Section